In [1]:
import netCDF4 as nc
import numpy as np

In [2]:
root = nc.Dataset('prod_latlon_merged_2017.nc')

In [3]:
root.dimensions

OrderedDict([('time',
              <class 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 12),
             ('longitude',
              <class 'netCDF4._netCDF4.Dimension'>: name = 'longitude', size = 181),
             ('latitude',
              <class 'netCDF4._netCDF4.Dimension'>: name = 'latitude', size = 51)])

In [4]:
root.variables['time']

<class 'netCDF4._netCDF4.Variable'>
int32 time(time)
    units: days since 1990-01-01T00:00:00Z
    long_name: julian day (UT)
    standard_name: time
    conventions: Relative julian days
    axis: T
unlimited dimensions: 
current shape = (12,)
filling on, default _FillValue of -2147483647 used

In [22]:
for name in root.ncattrs():
    print(name)

source
project
authors
contact
southernmost_latitude
northernmost_latitude
latitude_resolution
westernmost_longitude
easternmost_longitude
longitude_resolution
start_date
stop_date
field_type
sensor


In [37]:
root.variables['time'][2]

masked_array(data=9921,
             mask=False,
       fill_value=999999,
            dtype=int32)

In [13]:
ice_area = root.variables['ice_area']


array(nan, dtype=float32)

In [17]:
ice_area[1, -56, -72]

IndexError: Index out of range